In [ ]:
! pip install rouge_score evaluate nltk && pip install -U accelerate --quiet
! pip install -U transformers && pip install -U datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a8df1d4726922022459698563a87b7aacbfc93ed2263ebcb2a536b92739ddb34
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
import nltk

nltk.download("punkt")
nltk.download("punkt_tab")


ds = load_dataset("abisee/cnn_dailymail", "3.0.0")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

train_df = pd.DataFrame(ds["train"])
test_df = pd.DataFrame(ds["test"])

train_sample = train_df.sample(n=100, random_state=27)
test_sample = test_df.sample(n=50, random_state=27)

In [ ]:
first_example = train_sample.iloc[0]
print("Article:\n", first_example["article"])
print("\nRésumé:\n", first_example["highlights"])


print(train_sample.head())

print("\nAperçu échantillon test:")
print(test_sample.head())

Article:
 (CNN) -- A pregnant bartender says she is fighting to keep her job at a gentlemen's club in New York. Jennifer Paviglianiti, of Centereach, N.Y., claims her employer, Café Royale, discriminated against her because of her pregnancy. She filed a claim with the EEOC earlier this month. Paviglianiti says she feared how her boss would react to her pregnancy and wanted to wait a few months to tell him. However, John Doxey found out before she could tell him, and that's when Paviglianiti claims her job became threatened. Up until she became pregnant, the 29-year-old says she had been one of the club's most popular bartenders. Read the complaint . She says she "bonused 9 times" since August and it was only toward the end of her pregnancy that she asked for different hours. Due to the tough economy, Paviglianiti says she needed this specific bartending job and feared she'd soon be out of work. To protect herself, she decided to secretly record her boss on tape. On those recordings Dox

In [ ]:
def batch_generator(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]



In [ ]:
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer
import gc
def summarize_with_t5(texts, batch_size=8, model_name="t5-small"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"✅ Using device: {device}")

    # Chargement du modèle et du tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    summaries = []

    # Boucle par batchs
    for batch in batch_generator(texts, batch_size):
        # Préfixe pour le modèle T5
        prefixed_batch = ["summarize: " + text for text in batch]

        # Tokenization
        inputs = tokenizer(prefixed_batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

        # Génération
        summary_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=150,
            num_beams=4,
            early_stopping=True
        )

        # Décodage
        decoded_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        summaries.extend(decoded_summaries)

        # Libération de la mémoire
        del inputs, summary_ids
        torch.cuda.empty_cache()
        gc.collect()

    # Nettoyage final
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()

    return summaries


In [ ]:
texts_to_summarize = train_sample["article"].tolist()

# Générer les résumés avec T5-small
generated_summaries = summarize_with_t5(texts_to_summarize, batch_size=8, model_name="t5-small")

# Ajout au DataFrame
train_sample["generated_summary"] = generated_summaries

✅ Using device: cuda


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
print(train_sample[["article", "highlights", "generated_summary"]].head(5).T)
display(train_sample[["article", "highlights", "generated_summary"]].head(5))

                                                              10613   \
article            (CNN) -- A pregnant bartender says she is figh...   
highlights         Jennifer Paviglianiti says boss reduced hours,...   
generated_summary  a pregnant bartender says she is fighting to k...   

                                                              178307  \
article            By . Gemma Mullin . A burglar was caught red-h...   
highlights         Andres Balog, 26, triggered motion-sensor came...   
generated_summary  andres Balog, 26, forced his way into the prop...   

                                                              192807  \
article            An investigation into the deaths of five cance...   
highlights         Findings revealed after investigation into dea...   
generated_summary  keyhole surgery for stomach and throat cancer ...   

                                                              57522   \
article            (CNN) -- The French season is already up a

,article,highlights,generated_summary
10613,(CNN) -- A pregnant bartender says she is figh...,"Jennifer Paviglianiti says boss reduced hours,...",a pregnant bartender says she is fighting to k...
178307,By . Gemma Mullin . A burglar was caught red-h...,"Andres Balog, 26, triggered motion-sensor came...","andres Balog, 26, forced his way into the prop..."
192807,An investigation into the deaths of five cance...,Findings revealed after investigation into dea...,keyhole surgery for stomach and throat cancer ...
57522,(CNN) -- The French season is already up and r...,"Without Luis Suarez, Liverpool beat Southampto...",vanishing spray was cheered to the rafters by ...
44042,"(CNN) -- Meat is murder? Well, perhaps not for...",Ongoing research into in-vitro meat; grown in ...,in-vitro meat could be changing global diets w...


In [ ]:
def calculate_accuracy(generated, reference):
    correct = 0
    for g, r in zip(generated, reference):
        if g.strip().lower() == r.strip().lower():
            correct += 1
    return correct / len(generated)


In [ ]:
accuracy = calculate_accuracy(
    train_sample["generated_summary"],
    train_sample["highlights"]
)

#Normal que ce soit 0
print(f"Accuracy des résumés T5 (égalité exacte) : {accuracy:.4f}")

Accuracy des résumés T5 (égalité exacte) : 0.0000


In [ ]:
from nltk.tokenize import sent_tokenize
from evaluate import load

rouge = load("rouge")

def preprocess_summary(text):
    sentences = sent_tokenize(text)
    return "\n".join(sentences)

def compute_rouge_score(predictions, references, use_stemmer=True):
    predictions_processed = [preprocess_summary(p) for p in predictions]
    references_processed = [preprocess_summary(r) for r in references]

    return rouge.compute(
        predictions=predictions_processed,
        references=references_processed,
        use_stemmer=use_stemmer
    )

In [ ]:
rouge_scores_t5_small = compute_rouge_score(train_sample['highlights'], train_sample['generated_summary'])

print("ROUGE scores (t5-small):")
for k, v in rouge_scores_t5_small.items():
    print(f"{k}: {v:.4f}")

ROUGE scores (t5-small):
rouge1: 0.3675
rouge2: 0.1615
rougeL: 0.2697
rougeLsum: 0.3405
